<a href="https://colab.research.google.com/github/Tmiglin/Distracted-Drivers/blob/main/MSDS686_Final_Project_(Distracted_Drivers)_Miglin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statefarm Distracted Driver Detection 
According to the CDC motor vehicle safety divison, one in five car accidents are caused by a distracted driver. We have all heard this in many statefarm commericals but the question is if there is a way to classify a drivers behavior. Statefarm has sponsored a Kaggle competition to improve some worrying statistics provided by the CDC. This project will involve building a convulutional neural network using the functional API that will provide categorical image classification for distracted drivers. The network will build with categorical classification because I don't just want to know if the driver is distracted but what class of distraction they belong to (cell phone in hand, looking behind, hands off the wheel). Below I used Professor Sorauf's fit hack for convenience. 

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

## Here I am just import the libraries necessary to complete my task
#### *This is taken from the week 6 assignment since it proved to be what was necessary

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model, model_to_dot # This will print model architecture.
from tensorflow.keras.layers import Input, Concatenate, Dense, Dropout, Flatten, Activation # We add the Concatentate function
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, SeparableConv2D # We will use AveragePooling2D. Similar to MaxPooling but now we take the average value in the window.
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend
from cv2 import imread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display # Library to help view images
from PIL import Image # Library to help view images
from tensorflow.keras.preprocessing.image import ImageDataGenerator # Library for data augmentation
import os, shutil # Library for navigating files
from google.colab import drive # Library to mount google drives
np.random.seed(42)

### I downloaded the competition data to my google drive stream so I will be mounting the drive directory to this notebook. 

In [ ]:
drive.mount('/content/gdrive')

In [ ]:
# Specify the base directory where images are located.  You need to save your data here.
base_dir = '/content/gdrive/My Drive/Distracted_Driver/state-farm-distracted-driver-detection/imgs'

#### Before I explain the data it would be best to understand how the data will be split between train, test, and validation. The competition does not provide a validation data set, so instead I will be using a validation split to create a validation set for my models.

In [ ]:
# Specify the traning, validation, and test dirrectories.  
train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

## The Data
The data provided are images taken from dashboard cameras facing the driver. With the goal being to categorize what each driver is doing based on a certain amount of classes, the pictures can be of the same drivers doing different things. As seen in the below cells, the data set's provided can be separated into 10 different classes. Each class contains photos with attentive and non-attentive drivers, staying consistent to the class however. I've provided the labels for the class after taking a close look at an image from each c folder. I used these labels to fill in a dictionary of labels so that I could display each of the image classes below. 

In [ ]:
classes = [folder for folder in os.listdir(train_dir)]

# Description of picture class
class_dict = {
    'c0': 'safe driving',
    'c1': 'texting - right',
    'c2': 'talking on the phone - right',
    'c3': 'texting - left',
    'c4': 'talking on the phone - left',
    'c5': 'operating the radio',
    'c6': 'drinking',
    'c7': 'reaching behind',
    'c8': 'hair and makeup',
    'c9': 'talking to passenger'
}

In [ ]:
from cv2 import imread
for item in classes:
    path = os.path.join(train_dir,item)
    count=0
    for image in os.listdir(path):
        image_array = imread(os.path.join(path,image))
        count+=1
        print(item)
        print(class_dict[item])
        plt.imshow(image_array)
        plt.show()
        print('\n')
        break

# Data Augmentation
I am going to be using data augmentation to increase the diversity of the training data. This should improve the performance of my models. I won't be having too large of ranges to limit the time the model will take to train all of the data. I have 17,943 images to train the model on, 20% of those being used for validation. Then I have 4481 images to test my data on. 

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.1,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        fill_mode='nearest',
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='training')

test_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        subset='validation',
        shuffle=True)

# The First Model

This model is similar to the model used in week 5. I found this model architecture to be great for categorical image classfication after tooling around with it. I am normalizing after the convolution layers are merged together as well. The highest accuracy I saw from this model after 3 different runs was 93.4%, good accuracy but the training time was quite long.

* I made a light change to the model by adding another dense layer and it really heightened the accuracy so I will stick with this model instead

In [ ]:
visible = Input(shape=(224,224,3))

c1 = SeparableConv2D(64, (2,2), padding ='valid', activation = 'relu')(visible)

c2 = SeparableConv2D(128, (2,2), padding = 'valid', activation ='relu')(c1)
p1 = MaxPooling2D((2,2), padding = 'valid')(c2)

c3 = SeparableConv2D(128, (1,1), padding = 'valid', activation ='relu')(c1)
p2 = MaxPooling2D((2,2), padding = 'valid')(c3)

merge1 = Concatenate(axis=-1)([p1,p2])

norm1 = BatchNormalization()(merge1)

c4 = SeparableConv2D(256, (2,2), padding = 'valid', activation = 'relu')(norm1)
p3 = MaxPooling2D(2,2)(c4)

flat = Flatten()(p3)

hidden1 = Dense(512, activation = 'relu')(flat)
drop1 = Dropout(0.2)(hidden1)

hidden2 =Dense(256, activation = 'relu')(drop1)
drop2 = Dropout(0.2)(hidden2)

output = Dense(10, activation='softmax')(drop2)

model = Model(inputs=visible, outputs=output)

plot_model(model)

In [ ]:
model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

history = model.fit(train_generator,  
          epochs = 100,  
          validation_data=test_generator,
          verbose = 1,
          callbacks=[EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights = True)]
          ,use_multiprocessing=True) 

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

test_loss, test_acc = model.evaluate(test_generator)
print('test_acc:', test_acc)

# The Second Model
This model proved to be a success. This model follows a sort of AlexNet architecture with more hidden layers. Padding is consistent throughout the model as well as pooling. I stacked the convolutional layers with Normalization throughout as well. After 3 runs with this model the highest accuracy achieved sat at 93.7%, this model seems promising so now I will test it.

* This model will no longer be tested as the accuracy has dropped

In [ ]:
visible = Input(shape=(224,224,3))

c1 = Conv2D(64, (2,2), padding ='valid', activation = 'relu')(visible)
p1 = MaxPooling2D((2,2), padding = 'valid')(c1)

c2 = Conv2D(128, (2,2), padding = 'valid', activation ='relu')(c1)
p2 = MaxPooling2D((2,2), padding = 'valid')(c2)

c3 = Conv2D(256, (2,2), padding = 'valid', activation = 'relu')(p2)
p3 = MaxPooling2D((2,2), padding = 'valid')(c3)

flat = Flatten()(p3)

hidden1 = Dense(512, activation = 'relu')(flat)
drop1 = Dropout(0.2)(hidden1)

hidden2 = Dense(256, activation = 'relu')(drop1)
drop2 = Dropout(0.2)(hidden2)

output = Dense(10, activation='softmax')(drop2)

model = Model(inputs=visible, outputs=output)

plot_model(model)

In [ ]:
model.compile(optimizer = 'adam',
               loss = 'categorical_crossentropy',
               metrics = ['accuracy'])

history = model.fit(train_generator,  
          epochs = 100,  
          validation_data=test_generator,
          verbose = 1,
          callbacks=[EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights = True)]
          ,use_multiprocessing=True) 

history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(history_dict['accuracy']) + 1)

plt.plot(epochs, loss_values, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss_values, 'b', label = 'Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, acc_values, 'bo', label = 'Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

test_loss, test_acc = model.evaluate(test_generator)
print('test_acc:', test_acc)

In [ ]:
model.predict(test_generator, use_multiprocessing=True)

# Results Analysis
This model is success with great predictions coming from it. The training time for the models have been a bit tasking, but for this project I am quite happy with the results. Below I have found a nice function framework for displaying images next to the predictions and I will include it below for some visual flare.

In [ ]:
import cv2

batch = test_generator.next()
for i,j in zip(batch[0],batch[1]):
    plt.imshow(i)
    
    plt.show()
    print('Actual: ', class_dict['c'+str(np.argmax(j))])
    #print('\n')
    #print('\n')
    #break

    #image =cv2.resize(image,(224,224))
    image=i.reshape(1,224,224,3)
    print('predicted: ', class_dict['c'+ str(np.argmax(model.predict(image)))])
    print('\n')